In [ ]:
!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0


In [ ]:
from ortools.sat.python import cp_model

def planifier_examens(examens, salles, disponibilites, transition=1):

    model = cp_model.CpModel()

    # Variables de décision : X[e, j, h, s] = 1 si l'examen e est planifié le jour j, à l'heure h, en salle s
    X = {}
    for e in examens:
        for j in disponibilites:
            for h in disponibilites[j]:
                for s in salles:
                    X[e, j, h, s] = model.NewBoolVar(f'X_{e}_{j}_{h}_{s}')

    # Contrainte 1 : Chaque examen doit être programmé exactement une fois
    for e in examens:
        model.Add(sum(X[e, j, h, s] for j in disponibilites for h in disponibilites[j] for s in salles) == 1)

    # Contrainte 2 : Capacité des salles respectée
    for j in disponibilites:
        for h in disponibilites[j]:
            for s in salles:
                model.Add(
                    sum(X[e, j, h, s] * len(examens[e]["etudiants"]) for e in examens) <= salles[s]["capacite"]
                )

    # Contrainte 3 : Un étudiant ne peut pas avoir deux examens en même temps (y compris reprises et examens communs)
    for etudiant in set(sum([examens[e]["etudiants"] for e in examens], [])):
        for j in disponibilites:
            for h in disponibilites[j]:
                model.Add(
                    sum(X[e, j, h, s] for e in examens if etudiant in examens[e]["etudiants"] for s in salles) <= 1
                )

    # Contrainte 4 : Respect de la disponibilité des salles
    for j in disponibilites:
        for h in disponibilites[j]:
            for s in salles:
                if not disponibilites[j][h].get(s, True):  # Si la salle est indisponible
                    for e in examens:
                        model.Add(X[e, j, h, s] == 0)

    # Contrainte 5 : Durée variable des examens et marges de transition
    for e in examens:
        duree = examens[e]["duree"]
        for j in disponibilites:
            for h in disponibilites[j]:
                for s in salles:
                    for d in range(1, duree + transition):  # Bloquer les créneaux suivants
                        if h + d in disponibilites[j]:
                            model.Add(X[e, j, h, s] + sum(X[e, j, h + d, s] for e in examens) <= 1)

    # Objectif : Minimiser le temps total des examens (dernier examen programmé)
    dernier_instant = model.NewIntVar(0, 1000, 'dernier_instant')
    for e in examens:
        for j in disponibilites:
            for h in disponibilites[j]:
                for s in salles:
                    model.Add(dernier_instant >= h * X[e, j, h, s])
    model.Minimize(dernier_instant)

    # Résolution du problème
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        result = []
        for e in examens:
            for j in disponibilites:
                for h in disponibilites[j]:
                    for s in salles:
                        if solver.Value(X[e, j, h, s]) == 1:
                            result.append((e, j, h, s))
        return result
    else:
        return "Pas de solution trouvée"

# Exemple de données d'entrée
examens = {
    "Math": {"duree": 1, "etudiants": [1, 2, 3], "annee": 1, "reprise": False},
    "Physique": {"duree": 1, "etudiants": [2, 3, 4], "annee": 2, "reprise": True},
    "Info": {"duree": 2, "etudiants": [1, 4], "annee": 3, "reprise": True},
}

salles = {
    "A": {"capacite": 4},
    "B": {"capacite": 2},
}

disponibilites = {
    "Lundi": {8: {"A": True, "B": True}, 10: {"A": True, "B": False}},
    "Mardi": {8: {"A": True, "B": True}, 10: {"A": False, "B": True}},
}

# 📌 Exécution du solveur
planning = planifier_examens(examens, salles, disponibilites)
print("Planning généré :", planning)

Planning généré : [('Math', 'Mardi', 8, 'A'), ('Physique', 'Lundi', 10, 'A'), ('Info', 'Mardi', 10, 'B')]


In [ ]:
from ortools.sat.python import cp_model

def planifier_examens(examens, salles, disponibilites, transition=1):
    model = cp_model.CpModel()

    # Variables de décision : X[e, j, h, s] = 1 si l'examen e est planifié le jour j, à l'heure h, en salle s
    X = {}
    for e in examens:
        for j in disponibilites:
            for h in disponibilites[j]:
                for s in salles:
                    X[e, j, h, s] = model.NewBoolVar(f'X_{e}_{j}_{h}_{s}')

    # Contrainte 1 : Chaque examen doit être programmé exactement une fois
    for e in examens:
        model.Add(sum(X[e, j, h, s] for j in disponibilites for h in disponibilites[j] for s in salles) == 1)

    # Contrainte 2 : Une seule épreuve par salle et par créneau
    for j in disponibilites:
        for h in disponibilites[j]:
            for s in salles:
                model.Add(sum(X[e, j, h, s] for e in examens) <= 1)

    # Contrainte 3 : Capacité des salles respectée
    for j in disponibilites:
        for h in disponibilites[j]:
            for s in salles:
                model.Add(
                    sum(X[e, j, h, s] * len(examens[e]["etudiants"]) for e in examens) <= salles[s]["capacite"]
                )

    # Contrainte 4 : Un étudiant ne peut pas avoir deux examens en même temps
    for etudiant in set(sum([examens[e]["etudiants"] for e in examens], [])):
        for j in disponibilites:
            for h in disponibilites[j]:
                model.Add(
                    sum(X[e, j, h, s] for e in examens if etudiant in examens[e]["etudiants"] for s in salles) <= 1
                )

    # Contrainte 5 : Respect de la disponibilité des salles
    for j in disponibilites:
        for h in disponibilites[j]:
            for s in salles:
                if not disponibilites[j][h].get(s, True):  # Si la salle est indisponible
                    for e in examens:
                        model.Add(X[e, j, h, s] == 0)

    # Contrainte 6 : Durée des examens et respect de la transition
    for e in examens:
        duree = examens[e]["duree"]
        for j in disponibilites:
            for h in disponibilites[j]:
                for s in salles:
                    for d in range(1, duree + transition):  # Bloquer les créneaux suivants
                        if h + d in disponibilites[j]:
                            model.Add(X[e, j, h, s] + sum(X[e2, j, h + d, s] for e2 in examens if e2 != e) <= 1)

    # Objectif : Minimiser le dernier instant des examens
    dernier_instant = model.NewIntVar(0, max(max(disponibilites[j].keys()) for j in disponibilites), 'dernier_instant')
    for e in examens:
        for j in disponibilites:
            for h in disponibilites[j]:
                for s in salles:
                    model.Add(dernier_instant >= h).OnlyEnforceIf(X[e, j, h, s])
    model.Minimize(dernier_instant)

    # Résolution du problème
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        result = []
        for e in examens:
            for j in disponibilites:
                for h in disponibilites[j]:
                    for s in salles:
                        if solver.Value(X[e, j, h, s]) == 1:
                            result.append((e, j, h, s))
        return result
    else:
        return "Pas de solution trouvée"

# Exemple de données d'entrée
examens = {
    "Math": {"duree": 1, "etudiants": [1, 2, 3], "annee": 1, "reprise": False},
    "Physique": {"duree": 1, "etudiants": [2, 3, 4], "annee": 2, "reprise": True},
    "Info": {"duree": 2, "etudiants": [1, 4], "annee": 3, "reprise": True},
}

salles = {
    "A": {"capacite": 4},
    "B": {"capacite": 2},
}

disponibilites = {
    "Lundi": {8: {"A": True, "B": True}, 10: {"A": True, "B": False}},
    "Mardi": {8: {"A": True, "B": True}, 10: {"A": False, "B": True}},
}

# 📌 Exécution du solveur
planning = planifier_examens(examens, salles, disponibilites)
print("Planning généré :", planning)


Planning généré : [('Math', 'Lundi', 8, 'A'), ('Physique', 'Mardi', 8, 'A'), ('Info', 'Mardi', 10, 'B')]


In [10]:
from ortools.sat.python import cp_model

def planifier_examens(examens, salles, disponibilites, transition=1):
    model = cp_model.CpModel()

    # Variables de décision : X[e, j, h, s] = 1 si l'examen e est programmé le jour j, à l'heure h, en salle s
    X = {}
    for e in examens:
        for j in disponibilites:
            for h in disponibilites[j]:
                for s in salles:
                    X[e, j, h, s] = model.NewBoolVar(f'X_{e}_{j}_{h}_{s}')

    # Contrainte 1 : Chaque examen doit être programmé exactement une fois
    for e in examens:
        model.Add(sum(X[e, j, h, s] for j in disponibilites
                                  for h in disponibilites[j]
                                  for s in salles) == 1)

    # Contrainte 2 : Dans une même salle et au même créneau, plusieurs examens sont autorisés
    # seulement s'ils ont la même durée et proviennent de la même année.
    for j in disponibilites:
        for h in disponibilites[j]:
            for s in salles:
                for e1 in examens:
                    for e2 in examens:
                        if e1 < e2:  # éviter les doublons
                            # S'ils n'ont pas la même durée ou n'appartiennent pas à la même année,
                            # ils ne peuvent pas être dans la même salle au même créneau.
                            if (examens[e1]["duree"] != examens[e2]["duree"]) or (examens[e1]["annee"] != examens[e2]["annee"]):
                                model.Add(X[e1, j, h, s] + X[e2, j, h, s] <= 1)

    # Contrainte 3 : Capacité des salles respectée (en utilisant nb_etudiants)
    for j in disponibilites:
        for h in disponibilites[j]:
            for s in salles:
                model.Add(
                    sum(X[e, j, h, s] * examens[e]["nb_etudiants"] for e in examens)
                    <= salles[s]["capacite"]
                )

    # Contrainte 4 : Les examens de différentes années ne sont pas programmés simultanément,
    # même s'ils sont dans des salles différentes.
    for j in disponibilites:
        for h in disponibilites[j]:
            for e1 in examens:
                for e2 in examens:
                    if e1 < e2 and (examens[e1]["annee"] != examens[e2]["annee"]):
                        model.Add(
                            sum(X[e1, j, h, s] for s in salles) +
                            sum(X[e2, j, h, s] for s in salles)
                            <= 1
                        )

    # Contrainte 5 : Respect de la disponibilité des salles
    for j in disponibilites:
        for h in disponibilites[j]:
            for s in salles:
                if not disponibilites[j][h].get(s, True):  # salle indisponible
                    for e in examens:
                        model.Add(X[e, j, h, s] == 0)

    # Contrainte 6 : Transition entre examens dans la même salle (1h entre la fin d'un examen et le début d'un autre)
    for e in examens:
        duree = examens[e]["duree"]
        for j in disponibilites:
            for h in disponibilites[j]:
                for s in salles:
                    # Une fois que l'examen e démarre à h dans la salle s,
                    # les créneaux suivants, correspondant à sa durée + le temps de transition,
                    # doivent être "libres" dans cette salle.
                    for d in range(1, duree + transition):
                        if h + d in disponibilites[j]:
                            model.Add(
                                X[e, j, h, s] +
                                sum(X[e2, j, h + d, s] for e2 in examens if e2 != e)
                                <= 1
                            )

    # Objectif : Minimiser le dernier instant d'examen (pour réduire l'étalement total)
    dernier_instant = model.NewIntVar(0, max(max(disponibilites[j].keys()) for j in disponibilites), 'dernier_instant')
    for e in examens:
        for j in disponibilites:
            for h in disponibilites[j]:
                for s in salles:
                    model.Add(dernier_instant >= h).OnlyEnforceIf(X[e, j, h, s])
    model.Minimize(dernier_instant)

    # Résolution du problème
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        # Construction d'un planning hebdomadaire : dictionnaire jour -> liste d'examens triés par heure
        planning = {j: [] for j in disponibilites}
        for e in examens:
            for j in disponibilites:
                for h in disponibilites[j]:
                    for s in salles:
                        if solver.Value(X[e, j, h, s]) == 1:
                            planning[j].append((h, e, s, examens[e]["annee"], examens[e]["filiere"]))
        for j in planning:
            planning[j].sort(key=lambda x: x[0])
        return planning
    else:
        return "Pas de solution trouvée"

# Exemple de données d'entrée
examens = {
    "Math": {"duree": 1, "nb_etudiants": 30, "annee": "L1", "filiere": "Sciences"},
    "Physique": {"duree": 2, "nb_etudiants": 30, "annee": "L1", "filiere": "Sciences"},
    "Info": {"duree": 2, "nb_etudiants": 25, "annee": "L2", "filiere": "Informatique"},
    "Chimie": {"duree": 1, "nb_etudiants": 30, "annee": "L1", "filiere": "Chimie"},
}

salles = {
    "A": {"capacite": 60},
    "B": {"capacite": 30},
}

disponibilites = {
    "Lundi": {8: {"A": True, "B": True}, 10: {"A": True, "B": True}},
    "Mardi": {8: {"A": True, "B": True}, 10: {"A": True, "B": False}},
}

# Exécution du solveur
planning = planifier_examens(examens, salles, disponibilites)
print("Planning hebdomadaire généré :", planning)


Planning hebdomadaire généré : {'Lundi': [(8, 'Info', 'B', 'L2', 'Informatique')], 'Mardi': [(8, 'Math', 'A', 'L1', 'Sciences'), (8, 'Physique', 'B', 'L1', 'Sciences'), (8, 'Chimie', 'A', 'L1', 'Chimie')]}


In [11]:
from ortools.sat.python import cp_model

def is_available(disponibilites, day, room, hour):
    """Renvoie True si la salle 'room' est disponible à 'hour' le 'day',
       selon les intervalles d'ouverture définis dans disponibilites."""
    if room not in disponibilites.get(day, {}):
        return False
    for interval in disponibilites[day][room]:
        start, end = interval
        # On considère l'heure de début comme disponible, et l'heure 'end' comme non disponible.
        if start <= hour < end:
            return True
    return False

def planifier_examens(examens, salles, disponibilites, global_hours, transition=1):
    model = cp_model.CpModel()

    # Création des variables décisionnelles :
    # X[e, day, hour, room] = 1 si l'examen e est programmé le jour 'day' à l'heure 'hour' en salle 'room'
    X = {}
    for e in examens:
        for day in disponibilites:
            for hour in global_hours:
                for room in salles:
                    if is_available(disponibilites, day, room, hour):
                        X[e, day, hour, room] = model.NewBoolVar(f'X_{e}_{day}_{hour}_{room}')
                    else:
                        X[e, day, hour, room] = model.NewConstant(0)

    # Contrainte 1 : Chaque examen doit être programmé exactement une fois.
    for e in examens:
        model.Add(sum(X[e, day, hour, room]
                      for day in disponibilites
                      for hour in global_hours
                      for room in salles) == 1)

    # Contrainte 2 : Dans une même salle et au même créneau, plusieurs examens sont autorisés
    # seulement s'ils ont la même durée et proviennent de la même année.
    for day in disponibilites:
        for hour in global_hours:
            for room in salles:
                if is_available(disponibilites, day, room, hour):
                    for e1 in examens:
                        for e2 in examens:
                            if e1 < e2:
                                if (examens[e1]["duree"] != examens[e2]["duree"]) or (examens[e1]["annee"] != examens[e2]["annee"]):
                                    model.Add(X[e1, day, hour, room] + X[e2, day, hour, room] <= 1)

    # Contrainte 3 : Respect de la capacité des salles (en utilisant nb_etudiants)
    for day in disponibilites:
        for hour in global_hours:
            for room in salles:
                if is_available(disponibilites, day, room, hour):
                    model.Add(
                        sum(X[e, day, hour, room] * examens[e]["nb_etudiants"] for e in examens)
                        <= salles[room]["capacite"]
                    )

    # Contrainte 4 : Les examens de différentes années ne sont pas programmés simultanément,
    # même s'ils sont dans des salles différentes.
    for day in disponibilites:
        for hour in global_hours:
            for e1 in examens:
                for e2 in examens:
                    if e1 < e2 and (examens[e1]["annee"] != examens[e2]["annee"]):
                        model.Add(
                            sum(X[e1, day, hour, room] for room in salles) +
                            sum(X[e2, day, hour, room] for room in salles)
                            <= 1
                        )

    # Contrainte 5 : La disponibilité des salles est respectée (déjà prise en compte lors de la création de X)

    # Contrainte 6 : Transition entre examens dans la même salle.
    # Une fois qu'un examen démarre à 'hour' dans une salle, les créneaux suivants, correspondant à sa durée + transition,
    # doivent être libres dans cette même salle.
    for e in examens:
        duree = examens[e]["duree"]
        for day in disponibilites:
            for hour in global_hours:
                for room in salles:
                    if is_available(disponibilites, day, room, hour):
                        for d in range(1, duree + transition):
                            if hour + d in global_hours:
                                model.Add(
                                    X[e, day, hour, room] +
                                    sum(X[e2, day, hour + d, room] for e2 in examens if e2 != e)
                                    <= 1
                                )

    # Objectif : Minimiser le dernier instant d'examen (pour réduire l'étalement total)
    dernier_instant = model.NewIntVar(0, max(global_hours), 'dernier_instant')
    for e in examens:
        for day in disponibilites:
            for hour in global_hours:
                for room in salles:
                    if is_available(disponibilites, day, room, hour):
                        model.Add(dernier_instant >= hour).OnlyEnforceIf(X[e, day, hour, room])
    model.Minimize(dernier_instant)

    # Résolution du problème
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        planning = {day: [] for day in disponibilites}
        for e in examens:
            for day in disponibilites:
                for hour in global_hours:
                    for room in salles:
                        if solver.Value(X[e, day, hour, room]) == 1:
                            planning[day].append((hour, e, room, examens[e]["annee"], examens[e]["filiere"]))
        for day in planning:
            planning[day].sort(key=lambda x: x[0])
        return planning
    else:
        return "Pas de solution trouvée"

# Exemple de données d'entrée

examens = {
    "Math": {"duree": 1, "nb_etudiants": 30, "annee": "L1", "filiere": "Sciences"},
    "Physique": {"duree": 2, "nb_etudiants": 30, "annee": "L1", "filiere": "Sciences"},
    "Info": {"duree": 2, "nb_etudiants": 25, "annee": "L2", "filiere": "Informatique"},
    "Chimie": {"duree": 1, "nb_etudiants": 30, "annee": "L1", "filiere": "Chimie"},
}

salles = {
    "A": {"capacite": 60},
    "B": {"capacite": 30},
}

# Structure de disponibilités : jour -> salle -> liste d'intervalles (début, fin)
disponibilites = {
    "Lundi": {
        "A": [(8, 12), (13, 18)],  # salle A disponible de 8h à 12h et de 13h à 18h
        "B": [(9, 11), (15, 17)]
    },
    "Mardi": {
        "A": [(8, 13)],
        "B": [(10, 14)]
    }
}

global_hours = list(range(8, 19))  # Plage horaire globale de 8h à 18h

planning = planifier_examens(examens, salles, disponibilites, global_hours, transition=1)
print("Planning hebdomadaire généré :", planning)


Planning hebdomadaire généré : {'Lundi': [(8, 'Math', 'A', 'L1', 'Sciences'), (8, 'Chimie', 'A', 'L1', 'Chimie'), (9, 'Physique', 'B', 'L1', 'Sciences')], 'Mardi': [(8, 'Info', 'A', 'L2', 'Informatique')]}
